In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from gensim import corpora
from gensim.models import LdaModel
import pandas as pd


In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\night\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\night\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\night\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv(r'C:\Users\night\Downloads\npr.csv')  # Replace with the correct path if needed
documents = df['Article'].tolist()


In [4]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

preprocessed_documents = [preprocess_text(doc) for doc in documents]


In [5]:
dictionary = corpora.Dictionary(preprocessed_documents)
dictionary.filter_extremes(no_below=15, no_above=0.5)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_documents]


In [6]:
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)


In [7]:
article_labels = []
for doc in preprocessed_documents:
    bow = dictionary.doc2bow(doc)
    topics = lda_model.get_document_topics(bow)
    dominant_topic = max(topics, key=lambda x: x[1])[0]
    article_labels.append(dominant_topic)

df_result = pd.DataFrame({"Article": documents, "Topic": article_labels})
print(df_result)


                                                 Article  Topic
0      In the Washington of 2016, even when the polic...      4
1        Donald Trump has used Twitter  —   his prefe...      4
2        Donald Trump is unabashedly praising Russian...      1
3      Updated at 2:50 p. m. ET, Russian President Vl...      4
4      From photography, illustration and video, to d...      2
...                                                  ...    ...
11987  The number of law enforcement officers shot an...      4
11988    Trump is busy these days with victory tours,...      1
11989  It’s always interesting for the Goats and Soda...      0
11990  The election of Donald Trump was a surprise to...      1
11991  Voters in the English city of Sunderland did s...      1

[11992 rows x 2 columns]


In [8]:
for topic_id in range(lda_model.num_topics):
    print(f"Top terms for Topic #{topic_id}:")
    print([term[0] for term in lda_model.show_topic(topic_id, topn=10)])
    print()


Top terms for Topic #0:
['health', 'percent', 'school', 'study', 'company', 'care', 'state', 'student', 'child', 'program']

Top terms for Topic #1:
['trump', 'clinton', 'president', 'republican', 'state', 'campaign', 'election', 'vote', 'obama', 'voter']

Top terms for Topic #2:
['city', 'water', 'day', 'world', 'two', 'back', 'home', 'country', 'around', 'last']

Top terms for Topic #3:
['think', 'know', 'thing', 'life', 'woman', 'really', 'story', 'something', 'book', 'u']

Top terms for Topic #4:
['police', 'state', 'government', 'law', 'report', 'court', 'president', 'country', 'case', 'official']



In [9]:
for idx, topic in lda_model.print_topics():
    print(f"Topic {idx}:")
    terms = [term.strip() for term in topic.split("+")]
    for term in terms:
        weight, word = term.split("*")
        print(f"- {word.strip()} (weight: {weight.strip()})")
    print()


Topic 0:
- "health" (weight: 0.009)
- "percent" (weight: 0.006)
- "school" (weight: 0.006)
- "study" (weight: 0.005)
- "company" (weight: 0.005)
- "care" (weight: 0.005)
- "state" (weight: 0.005)
- "student" (weight: 0.004)
- "child" (weight: 0.004)
- "program" (weight: 0.004)

Topic 1:
- "trump" (weight: 0.034)
- "clinton" (weight: 0.014)
- "president" (weight: 0.010)
- "republican" (weight: 0.010)
- "state" (weight: 0.009)
- "campaign" (weight: 0.009)
- "election" (weight: 0.007)
- "vote" (weight: 0.006)
- "obama" (weight: 0.006)
- "voter" (weight: 0.006)

Topic 2:
- "city" (weight: 0.005)
- "water" (weight: 0.004)
- "day" (weight: 0.004)
- "world" (weight: 0.004)
- "two" (weight: 0.003)
- "back" (weight: 0.003)
- "home" (weight: 0.003)
- "country" (weight: 0.003)
- "around" (weight: 0.003)
- "last" (weight: 0.003)

Topic 3:
- "think" (weight: 0.006)
- "know" (weight: 0.006)
- "thing" (weight: 0.006)
- "life" (weight: 0.005)
- "woman" (weight: 0.005)
- "really" (weight: 0.005)
- "sto